# OpenAI_Value1

**[Prompt]**
* Provide the value list
* Reqeust to identify 1-3 values included in each sentence


**[Input Sentence]**
* Conclusion

In [ ]:
# Install openai package to use OpenAI APIs
!pip install openai

# OpenAI API Key
# [WARNING] Once you hard-code your key here, keep this notebook private.
# If anyone uses this key, you'll get charged. If you believe your API key has
# been leaked, revoke the key on the OpenAI dashboard immediately.

### Upload Files

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
import pandas as pd
arguments = pd.read_csv('/content/gdrive/MyDrive/Value/arguments-test100.tsv', sep='\t')
labels = pd.read_csv('/content/gdrive/MyDrive/Value/labels-test100.tsv', sep='\t')

### Value ChatGPT

In [ ]:
# Value ChatGPT class
from openai import OpenAI

client = OpenAI (
api_key = "")

class ValueChatGPT:
  def __init__(self, client, max_tokens=100):
    client=client
    self.max_tokens = max_tokens
    self.messages = []

  def call(self):
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=self.messages,
      max_tokens=self.max_tokens,
    )
    return response

  def prompt(self, text):
    # Provide some character in the first user prompt
    user_prompt = ""
    if len(self.messages) == 0:
        user_prompt += (
            "Values are in the list: ['Self-direction: thought', 'Self-direction: action', 'Stimulation', 'Hedonism', 'Achievement', 'Power: dominance', 'Power: resources', 'Face', 'Security: personal', 'Security: societal', 'Tradition', 'Conformity: rules', 'Conformity: interpersonal', 'Humility', 'Benevolence: caring', 'Benevolence: dependability', 'Universalism: concern', 'Universalism: nature', 'Universalism: tolerance', 'Universalism: objectivity']"
            "Can you tell me what values are in the following input sentence? You can choose between 1 to 3 values in the above list."
            "Just give me answer of the values in [], do not respond in sentence.\n\n"
        )
    user_prompt += text

    # Update the messages so far
    self.messages.append({
        "role": "user",
        "content": user_prompt,
    })

    # Call ChatGPT
    response = self.call()

    # Save the returned message
    message = response.choices[0].message
    self.messages.append(message)
    self.messages=[]

    return message.content

In [ ]:
# Run an interactive console to chat with ChatGPT
labels_list=[]
def run_console(chatgpt):

  for i in range (len(arguments)):

    # Receive user input
    prompt = arguments.iloc[i,1]

    # Receive and display ChatGPT response
    response = chatgpt.prompt(prompt)
    labels_list.append([response])
    print("ChatGPT:", response)
    print(len(labels_list))

In [ ]:
# Make a ChatGPT object
value_chatgpt = ValueChatGPT(client)

run_console(value_chatgpt)

### Result into DataFrame

In [ ]:
# Make the dataframe filled with 0
columns=['Argument ID','Self-direction: thought','Self-direction: action','Stimulation','Hedonism','Achievement',
 'Power: dominance','Power: resources','Face','Security: personal','Security: societal','Tradition','Conformity: rules',
 'Conformity: interpersonal','Humility','Benevolence: caring','Benevolence: dependability','Universalism: concern',
 'Universalism: nature','Universalism: tolerance','Universalism: objectivity']

value_result = pd.DataFrame(index=range(0,100), columns = columns)
value_result = value_result.fillna(0)

In [ ]:
# Change the value into 1 if the value is in the labels_list
for i in range(len(columns)):
    for j in range(len(labels_list)):
        if columns[i] in str(labels_list[j]):
            value_result.iloc[j, i] = 1

In [ ]:
# Save the results in csv file
value_result.to_csv('/content/gdrive/MyDrive/Value/table.csv')

### Calculate Scores

In [ ]:
# Drop Argument ID to calculate scores
value_result = value_result.drop(['Argument ID'], axis=1)
labels = labels.drop(['Argument ID'], axis=1)

In [ ]:
# Calculate numbers of TP, FP, TN, FN
# Labels is original and value_result is the result
True_Positive=0
False_Positive=0
True_Negative=0
False_Negative=0

for i in range (len(value_result)):
  for j in range (len(columns)-1):
    if value_result.iloc[i,j]==1 and labels.iloc[i,j]==1:
      True_Positive+=1
    if value_result.iloc[i,j]==1 and labels.iloc[i,j]==0:
      False_Positive+=1
    if value_result.iloc[i,j]==0 and labels.iloc[i,j]==1:
      False_Negative+=1
    if value_result.iloc[i,j]==0 and labels.iloc[i,j]==0:
      True_Negative+=1

In [ ]:
# Calculate Precision, Recall, F1 Score
Precision = True_Positive / (True_Positive + False_Positive)
Recall = True_Positive / (True_Positive + False_Negative)
F1_Score = 2*Precision*Recall / (Precision+Recall)

# Print the scores
print("Precision:", Precision)
print("Recall:", Recall)
print("F1 Score:", F1_Score)